<a href="https://colab.research.google.com/github/MahmoudEssam707/Data-Tools/blob/main/Iris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
# Import necessary libraries
import pandas as pd
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib
matplotlib.use('Agg')
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [67]:
def wrangle(iris):
    # Reading Data
    iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
    # Column with species names
    iris_df['species'] = pd.Categorical.from_codes(iris.target, iris.target_names)
    # Encoded Columns
    iris_df['encoded'] = iris.target
    # Let's make sure of intervals for data
    print(iris_df.describe(),'_'*100,sep='\n')
    #checking duplicated data and deleting it
    print(f"Number of Duplications in Iris Data = {iris_df.duplicated().sum()}\n{'_'*100}","Voilà, now there's no duplication in data :3",sep='\n')
    iris_df.drop_duplicates(inplace=True)
    # Let's See Nulls
    print(iris_df.isnull().sum(),'_'*100,sep='\n')
    return iris_df


In [68]:
# Load iris dataset
iris = load_iris()
iris_df = wrangle(iris=iris)
iris_df

       sepal length (cm)  sepal width (cm)  petal length (cm)  \
count         150.000000        150.000000         150.000000   
mean            5.843333          3.057333           3.758000   
std             0.828066          0.435866           1.765298   
min             4.300000          2.000000           1.000000   
25%             5.100000          2.800000           1.600000   
50%             5.800000          3.000000           4.350000   
75%             6.400000          3.300000           5.100000   
max             7.900000          4.400000           6.900000   

       petal width (cm)     encoded  
count        150.000000  150.000000  
mean           1.199333    1.000000  
std            0.762238    0.819232  
min            0.100000    0.000000  
25%            0.300000    0.000000  
50%            1.300000    1.000000  
75%            1.800000    2.000000  
max            2.500000    2.000000  
________________________________________________________________________

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species,encoded
0,5.1,3.5,1.4,0.2,setosa,0
1,4.9,3.0,1.4,0.2,setosa,0
2,4.7,3.2,1.3,0.2,setosa,0
3,4.6,3.1,1.5,0.2,setosa,0
4,5.0,3.6,1.4,0.2,setosa,0
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica,2
146,6.3,2.5,5.0,1.9,virginica,2
147,6.5,3.0,5.2,2.0,virginica,2
148,6.2,3.4,5.4,2.3,virginica,2


In [69]:
# Scatter Plot
sns.pairplot(iris_df, hue='species', markers=["o", "s", "D"], palette='viridis')
plt.title('Scatter Plot of Iris Dataset')
plt.show()

# Box Plot
fig, axes = plt.subplots(2, 2, figsize=(10, 10))
for i, ax in enumerate(axes.flatten()):
    sns.boxplot(x='species', y=iris.feature_names[i], data=iris_df, ax=ax)
plt.tight_layout()
plt.show()

# Heatmap
species_names = iris.target_names
species_corr = iris_df.corr()
plt.figure(figsize=(10, 10))
sns.heatmap(species_corr, annot=True, cmap='coolwarm', xticklabels=species_names, yticklabels=species_names)
plt.title('Correlation Heatmap')
plt.show()

In [70]:
X = iris_df.drop(columns=['encoded','species'])
y = iris_df['encoded']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [71]:
# Create a Decision Tree classifier
model = DecisionTreeClassifier()

# Define the hyperparameters to tune
param_grid = {
    'max_depth': [3, 5, 7, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Perform Grid Search for hyperparameter tuning
grid_search = GridSearchCV(model, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Get the best parameters and the best score
best_model = grid_search.best_estimator_


In [77]:
# Create a Sequential model
Neural = Sequential()
Neural.add(Dense(8, input_dim=4, activation='relu'))
Neural.add(Dense(8, activation='relu'))
Neural.add(Dense(3, activation='softmax'))

# Compile the model
Neural.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
Neural.fit(X_train, y_train, epochs=100, batch_size=5, verbose=0)

In [78]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

def perform_prediction(sepal_length, sepal_width, petal_length, petal_width, model_type):
    if model_type == 'Decision Tree':
        user_input = [[sepal_length, sepal_width, petal_length, petal_width]]
        predicted_species = best_model.predict(user_input)
        species_name = iris.target_names[predicted_species[0]]
        print(f"The predicted species for the given values is: {species_name}")
    else:
        # Here, 'neural_network_model' should represent your trained neural network model
        user_input = [[sepal_length, sepal_width, petal_length, petal_width]]
        predicted_probabilities = Neural.predict(np.array(user_input))
        predicted_species = np.argmax(predicted_probabilities)
        species_name = iris.target_names[predicted_species]
        print(f"The predicted species for the given values is: {species_name}")

# Define input widgets
sepal_length = widgets.FloatSlider(value=5.1, min=0, max=10.0, step=0.1)
sepal_width = widgets.FloatSlider(value=3.5, min=0, max=10.0, step=0.1)
petal_length = widgets.FloatSlider(value=1.4, min=0, max=10.0, step=0.1)
petal_width = widgets.FloatSlider(value=0.2, min=0, max=10.0, step=0.1)
model = widgets.Dropdown(options=['Decision Tree', 'Neural Network'], description='Model')

# Create interactive UI
interactive_plot = interactive(perform_prediction, sepal_length=sepal_length, sepal_width=sepal_width,
                               petal_length=petal_length, petal_width=petal_width, model_type=model)

# Display the UI
display(interactive_plot)


interactive(children=(FloatSlider(value=5.1, description='sepal_length', max=10.0), FloatSlider(value=3.5, des…